In [28]:
import psycopg2
import json
from datetime import datetime
import os
from dotenv import load_dotenv
import pandas as pd

In [29]:
DB_CONFIG = {
    'host': 'localhost',
    'database': 'mates',
    'user': 'postgres',
    'password': '1',
    'port': '5432'
}

CONNECTION_STRING = "postgresql://postgres:1@localhost:5432/mates"
JSON_FILE_PATH = r"d:\Menghour\MATES\scraping\notebook\khmer_news_formatted.json"

In [30]:
# Create normalized database tables
def create_normalized_tables(conn):
    """Create all normalized tables"""
    create_tables_sql = """
    -- Drop tables if they exist (for clean setup)
    DROP TABLE IF EXISTS article_tags CASCADE;
    DROP TABLE IF EXISTS articles CASCADE;
    DROP TABLE IF EXISTS tags CASCADE;
    DROP TABLE IF EXISTS sources CASCADE;
    DROP TABLE IF EXISTS categories CASCADE;

    -- Create categories table
    CREATE TABLE categories (
        category_id SERIAL PRIMARY KEY,
        category_name VARCHAR(100) UNIQUE NOT NULL,
        description TEXT,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );

    -- Create sources table
    CREATE TABLE sources (
        source_id SERIAL PRIMARY KEY,
        source_url VARCHAR(500) UNIQUE NOT NULL,
        source_name VARCHAR(200) NOT NULL,
        is_active BOOLEAN DEFAULT TRUE,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );

    -- Create tags table
    CREATE TABLE tags (
        tag_id SERIAL PRIMARY KEY,
        tag_name VARCHAR(100) UNIQUE NOT NULL,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );

    -- Create articles table
    CREATE TABLE articles (
        article_id SERIAL PRIMARY KEY,
        url VARCHAR(1000) UNIQUE NOT NULL,
        source_id INTEGER REFERENCES sources(source_id),
        publication_date DATE NOT NULL,
        scrape_date TIMESTAMP NOT NULL,
        title TEXT NOT NULL,
        content TEXT,
        word_count INTEGER DEFAULT 0,
        sentence_count INTEGER DEFAULT 0,
        character_count INTEGER DEFAULT 0,
        category_id INTEGER REFERENCES categories(category_id),
        category_confidence DECIMAL(3,2) DEFAULT 0.0,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );

    -- Create article_tags junction table
    CREATE TABLE article_tags (
        article_tag_id SERIAL PRIMARY KEY,
        article_id INTEGER REFERENCES articles(article_id) ON DELETE CASCADE,
        tag_id INTEGER REFERENCES tags(tag_id) ON DELETE CASCADE,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        UNIQUE(article_id, tag_id)
    );

    -- Create indexes for better performance
    CREATE INDEX idx_articles_url ON articles(url);
    CREATE INDEX idx_articles_publication_date ON articles(publication_date);
    CREATE INDEX idx_articles_source_id ON articles(source_id);
    CREATE INDEX idx_articles_category_id ON articles(category_id);
    CREATE INDEX idx_articles_created_at ON articles(created_at);
    CREATE INDEX idx_article_tags_article_id ON article_tags(article_id);
    CREATE INDEX idx_article_tags_tag_id ON article_tags(tag_id);
    CREATE INDEX idx_sources_url ON sources(source_url);
    CREATE INDEX idx_categories_name ON categories(category_name);
    CREATE INDEX idx_tags_name ON tags(tag_name);
    """
    
    try:
        cur = conn.cursor()
        cur.execute(create_tables_sql)
        conn.commit()
        cur.close()
        print("Normalized tables created successfully!")
        return True
    except Exception as e:
        print(f"Table creation failed: {e}")
        return False

In [31]:
# Test database connection and create tables
def test_connection():
    """Test the database connection"""
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        print("Database connection successful!")
        
        # Create normalized tables
        create_normalized_tables(conn)
        
        conn.close()
        return True
    except Exception as e:
        print(f"Database connection failed: {e}")
        return False

# Test the connection
test_connection()

Database connection successful!
Normalized tables created successfully!


True

In [32]:
# Load and preview JSON data
def load_json_data(file_path):
    """Load and preview JSON data"""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
        
        print(f"Successfully loaded {len(data)} articles")
        print("\nSample of first article:")
        print(f"Title: {data[0]['title'][:100]}...")
        print(f"URL: {data[0]['url']}")
        print(f"Source: {data[0]['source']}")
        print(f"Publication Date: {data[0]['publication_date']}")
        print(f"Primary Category: {data[0]['primary_category']}")
        print(f"Tags: {data[0]['tags'][:5]}")  # First 5 tags
        
        return data
    except Exception as e:
        print(f"Failed to load JSON file: {e}")
        return []
    
# Load your data
articles_data = load_json_data(JSON_FILE_PATH)

# Show basic statistics
if articles_data:
    print(f"\nData Overview:")
    print(f"Total articles: {len(articles_data)}")
    print(f"Unique sources: {len(set(article['source'] for article in articles_data))}")
    print(f"Unique categories: {len(set(article['primary_category'] for article in articles_data))}")
    print(f"Total unique tags: {len(set(tag for article in articles_data for tag in article.get('tags', [])))}")
    print(f"Date range: {min(article['publication_date'] for article in articles_data)} to {max(article['publication_date'] for article in articles_data)}")

Successfully loaded 7856 articles

Sample of first article:
Title: ចំនួននៃការវាយប្រហារ ទៅលើពលរដ្ឋរុស្ស៊ី នៅក្នុងប្រទេសអាល្លឺម៉ង់ កើនឡើងជារៀងរាល់ថ្ងៃ...
URL: https://dap-news.com/international/2022/03/29/232389/
Source: https://dap-news.com
Publication Date: 03/29/2022
Failed to load JSON file: 'primary_category'


In [33]:
# ETL Processor for Normalized Database
class NormalizedETL:
    def __init__(self, db_config):
        self.db_config = db_config
        self.conn = None
        self.cache = {
            'sources': {},
            'categories': {},
            'tags': {}
        }
    
    def connect(self):
        """Establish database connection"""
        try:
            self.conn = psycopg2.connect(**self.db_config)
            return True
        except Exception as e:
            print(f"Database connection failed: {e}")
            return False
    
    def load_cache(self):
        """Load existing data into cache to avoid duplicates"""
        try:
            cur = self.conn.cursor()
            
            # Load sources
            cur.execute("SELECT source_id, source_url FROM sources")
            for source_id, source_url in cur.fetchall():
                self.cache['sources'][source_url] = source_id
            
            # Load categories
            cur.execute("SELECT category_id, category_name FROM categories")
            for category_id, category_name in cur.fetchall():
                self.cache['categories'][category_name] = category_id
            
            # Load tags
            cur.execute("SELECT tag_id, tag_name FROM tags")
            for tag_id, tag_name in cur.fetchall():
                self.cache['tags'][tag_name] = tag_id
            
            cur.close()
            print("Cache loaded successfully")
        except Exception as e:
            print(f"Cache loading failed: {e}")
    
    def get_or_create_source(self, source_url, source_name):
        """Get existing source ID or create new source"""
        if source_url in self.cache['sources']:
            return self.cache['sources'][source_url]
        
        try:
            cur = self.conn.cursor()
            cur.execute(
                "INSERT INTO sources (source_url, source_name) VALUES (%s, %s) RETURNING source_id",
                (source_url, source_name)
            )
            source_id = cur.fetchone()[0]
            self.conn.commit()
            self.cache['sources'][source_url] = source_id
            cur.close()
            return source_id
        except Exception as e:
            print(f"Failed to create source {source_url}: {e}")
            self.conn.rollback()
            return None
    
    def get_or_create_category(self, category_name):
        """Get existing category ID or create new category"""
        if category_name in self.cache['categories']:
            return self.cache['categories'][category_name]
        
        try:
            cur = self.conn.cursor()
            cur.execute(
                "INSERT INTO categories (category_name) VALUES (%s) RETURNING category_id",
                (category_name,)
            )
            category_id = cur.fetchone()[0]
            self.conn.commit()
            self.cache['categories'][category_name] = category_id
            cur.close()
            return category_id
        except Exception as e:
            print(f"Failed to create category {category_name}: {e}")
            self.conn.rollback()
            return None
    
    def get_or_create_tag(self, tag_name):
        """Get existing tag ID or create new tag"""
        if tag_name in self.cache['tags']:
            return self.cache['tags'][tag_name]
        
        try:
            cur = self.conn.cursor()
            cur.execute(
                "INSERT INTO tags (tag_name) VALUES (%s) RETURNING tag_id",
                (tag_name,)
            )
            tag_id = cur.fetchone()[0]
            self.conn.commit()
            self.cache['tags'][tag_name] = tag_id
            cur.close()
            return tag_id
        except Exception as e:
            print(f"Failed to create tag {tag_name}: {e}")
            self.conn.rollback()
            return None
    
    def transform_article(self, article):
        """Transform a single article"""
        try:
            # Convert dates
            publication_date = datetime.strptime(article['publication_date'], '%m/%d/%Y').date()
            scrape_date = datetime.strptime(article['scrape_date'], '%m/%d/%Y %H:%M')
            
            transformed = {
                'url': article['url'],
                'source_url': article['source'],
                'source_name': article['source'].replace('https://', '').replace('http://', '').split('/')[0],
                'publication_date': publication_date,
                'scrape_date': scrape_date,
                'title': article['title'],
                'content': article.get('content', ''),
                'tags': article.get('tags', []),
                'word_count': article.get('word_count', 0),
                'sentence_count': article.get('sentence_count', 0),
                'character_count': article.get('character_count', 0),
                'primary_category': article.get('primary_category', 'unknown'),
                'category_confidence': float(article.get('category_confidence', 0.0))
            }
            
            return transformed
        except Exception as e:
            print(f"Transformation failed for {article.get('url', 'unknown')}: {e}")
            return None
    
    def process_article(self, transformed_article):
        """Process a single article into normalized tables"""
        try:
            cur = self.conn.cursor()
            
            # Get or create source
            source_id = self.get_or_create_source(
                transformed_article['source_url'],
                transformed_article['source_name']
            )
            if not source_id:
                return False
            
            # Get or create category
            category_id = self.get_or_create_category(transformed_article['primary_category'])
            if not category_id:
                return False
            
            # Insert article
            article_query = """
            INSERT INTO articles (
                url, source_id, publication_date, scrape_date, title, content,
                word_count, sentence_count, character_count, category_id, category_confidence
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT (url) DO UPDATE SET
                title = EXCLUDED.title,
                content = EXCLUDED.content,
                word_count = EXCLUDED.word_count,
                category_id = EXCLUDED.category_id,
                category_confidence = EXCLUDED.category_confidence
            RETURNING article_id
            """
            
            cur.execute(article_query, (
                transformed_article['url'],
                source_id,
                transformed_article['publication_date'],
                transformed_article['scrape_date'],
                transformed_article['title'],
                transformed_article['content'],
                transformed_article['word_count'],
                transformed_article['sentence_count'],
                transformed_article['character_count'],
                category_id,
                transformed_article['category_confidence']
            ))
            
            article_id = cur.fetchone()[0]
            
            # Process tags
            for tag_name in transformed_article['tags']:
                tag_id = self.get_or_create_tag(tag_name)
                if tag_id:
                    try:
                        cur.execute(
                            "INSERT INTO article_tags (article_id, tag_id) VALUES (%s, %s) ON CONFLICT DO NOTHING",
                            (article_id, tag_id)
                        )
                    except Exception as e:
                        print(f"Failed to link tag {tag_name} to article: {e}")
            
            self.conn.commit()
            cur.close()
            return True
            
        except Exception as e:
            print(f"Failed to process article {transformed_article['url']}: {e}")
            self.conn.rollback()
            return False
    
    def run_etl(self, json_file_path, batch_size=50):
        """Run the complete ETL process"""
        print("Starting Normalized ETL Process...")
        
        if not self.connect():
            return
        
        # Load cache of existing data
        self.load_cache()
        
        # Step 1: Extract
        print("Step 1: Extracting data from JSON...")
        with open(json_file_path, 'r', encoding='utf-8') as file:
            raw_articles = json.load(file)
        
        print(f"Loaded {len(raw_articles)} articles")
        
        # Step 2: Transform
        print("Step 2: Transforming data...")
        transformed_articles = []
        failed_transformations = 0
        
        for article in raw_articles:
            transformed = self.transform_article(article)
            if transformed:
                transformed_articles.append(transformed)
            else:
                failed_transformations += 1
        
        print(f"Successfully transformed: {len(transformed_articles)}")
        print(f"Failed transformations: {failed_transformations}")
        
        # Step 3: Load
        print("Step 3: Loading data to normalized tables...")
        
        success_count = 0
        error_count = 0
        
        # Process in batches
        for i in range(0, len(transformed_articles), batch_size):
            batch = transformed_articles[i:i + batch_size]
            batch_success = 0
            batch_error = 0
            
            for article in batch:
                if self.process_article(article):
                    batch_success += 1
                else:
                    batch_error += 1
            
            success_count += batch_success
            error_count += batch_error
            
            print(f"Batch {i//batch_size + 1}/{(len(transformed_articles) + batch_size - 1)//batch_size}: {batch_success} successful, {batch_error} failed")
        
        self.conn.close()
        
        print(f"ETL Complete!")
        print(f"Final Results: {success_count} successful, {error_count} failed")

# Run the normalized ETL process
etl = NormalizedETL(DB_CONFIG)
etl.run_etl(JSON_FILE_PATH)

Starting Normalized ETL Process...
Cache loaded successfully
Step 1: Extracting data from JSON...
Loaded 7856 articles
Step 2: Transforming data...
Successfully transformed: 7856
Failed transformations: 0
Step 3: Loading data to normalized tables...
Batch 1/158: 50 successful, 0 failed
Batch 2/158: 50 successful, 0 failed
Batch 3/158: 50 successful, 0 failed
Batch 4/158: 50 successful, 0 failed
Batch 5/158: 50 successful, 0 failed
Batch 6/158: 50 successful, 0 failed
Batch 7/158: 50 successful, 0 failed
Batch 8/158: 50 successful, 0 failed
Batch 9/158: 50 successful, 0 failed
Batch 10/158: 50 successful, 0 failed
Batch 11/158: 50 successful, 0 failed
Batch 12/158: 50 successful, 0 failed
Batch 13/158: 50 successful, 0 failed
Batch 14/158: 50 successful, 0 failed
Batch 15/158: 50 successful, 0 failed
Batch 16/158: 50 successful, 0 failed
Batch 17/158: 50 successful, 0 failed
Batch 18/158: 50 successful, 0 failed
Batch 19/158: 50 successful, 0 failed
Batch 20/158: 50 successful, 0 failed

In [34]:
# Verify the data was loaded into normalized tables
def verify_normalized_data():
    """Verify that data was successfully loaded into normalized tables"""
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cur = conn.cursor()
        
        # Count records in each table
        cur.execute("SELECT COUNT(*) FROM articles")
        article_count = cur.fetchone()[0]
        
        cur.execute("SELECT COUNT(*) FROM sources")
        source_count = cur.fetchone()[0]
        
        cur.execute("SELECT COUNT(*) FROM categories")
        category_count = cur.fetchone()[0]
        
        cur.execute("SELECT COUNT(*) FROM tags")
        tag_count = cur.fetchone()[0]
        
        cur.execute("SELECT COUNT(*) FROM article_tags")
        article_tag_count = cur.fetchone()[0]
        
        # Get some sample data with joins
        cur.execute("""
            SELECT a.title, s.source_name, c.category_name, a.publication_date
            FROM articles a
            JOIN sources s ON a.source_id = s.source_id
            JOIN categories c ON a.category_id = c.category_id
            ORDER BY a.publication_date DESC 
            LIMIT 5
        """)
        sample_articles = cur.fetchall()
        
        # Get category distribution
        cur.execute("""
            SELECT c.category_name, COUNT(*) as article_count
            FROM articles a
            JOIN categories c ON a.category_id = c.category_id
            GROUP BY c.category_name 
            ORDER BY article_count DESC
        """)
        category_stats = cur.fetchall()
        
        # Get top tags
        cur.execute("""
            SELECT t.tag_name, COUNT(*) as usage_count
            FROM article_tags at
            JOIN tags t ON at.tag_id = t.tag_id
            GROUP BY t.tag_name
            ORDER BY usage_count DESC
            LIMIT 10
        """)
        top_tags = cur.fetchall()
        
        cur.close()
        conn.close()
        
        print(f"Verification Complete!")
        print(f"\nTable Statistics:")
        print(f"   Articles: {article_count}")
        print(f"   Sources: {source_count}")
        print(f"   Categories: {category_count}")
        print(f"   Tags: {tag_count}")
        print(f"   Article-Tag relationships: {article_tag_count}")
        
        print(f"\nLatest 5 articles:")
        for i, (title, source, category, date) in enumerate(sample_articles, 1):
            print(f"   {i}. {title[:60]}...")
            print(f"      Source: {source} | Category: {category} | Date: {date}")
        
        print(f"\nCategory distribution:")
        for category, count in category_stats:
            print(f"   {category}: {count} articles")
        
        print(f"\nTop 10 tags:")
        for tag, count in top_tags:
            print(f"   {tag}: {count} articles")
            
    except Exception as e:
        print(f"Verification failed: {e}")

# Run verification
verify_normalized_data()

Verification Complete!

Table Statistics:
   Articles: 7856
   Sources: 1
   Categories: 1
   Tags: 0
   Article-Tag relationships: 0

Latest 5 articles:
   1. របស់រុស្ស៊ីថា ចារកម្មអ៊ុយក្រែន និងអង់គ្លេស ព្យាយាមសូកអ្នក បើ...
      Source: dap-news.com | Category: unknown | Date: 2025-11-11
   2. មេអាវុធហត្ថខេត្តកំពង់ស្ពឺ ជម្រុញឱ្យមន្ត្រីជំនាញបង្កើនការចុះផ...
      Source: dap-news.com | Category: unknown | Date: 2025-11-11
   3. មន្ទីរអភិវឌ្ឍន៍ជនបទ ប្រគល់ភារកិច្ចជូនអាជ្ញាធរភូមិឃុំនិងស្រុក...
      Source: dap-news.com | Category: unknown | Date: 2025-11-11
   4. លោក ខូយ រីដា អញ្ជើញចុះពិនិត្យ វឌ្ឍនភាពការងារសាង់សង់គម្រោងវារ...
      Source: dap-news.com | Category: unknown | Date: 2025-11-11
   5. លោកឧត្តមសេនីយ៍ត្រី ម៉េង ស្រ៊ុន មេបញ្ជាការ បានអញ្ជើញជាអធិបតីក...
      Source: dap-news.com | Category: unknown | Date: 2025-11-11

Category distribution:
   unknown: 7856 articles

Top 10 tags:


In [35]:
# Create a comprehensive dashboard
def create_normalized_dashboard():
    """Create a comprehensive dashboard showing normalized data"""
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        
        # Load data into pandas DataFrames
        articles_df = pd.read_sql("""
            SELECT 
                a.title,
                s.source_name,
                c.category_name,
                a.publication_date,
                a.word_count,
                a.scrape_date
            FROM articles a
            JOIN sources s ON a.source_id = s.source_id
            JOIN categories c ON a.category_id = c.category_id
        """, conn)
        
        tags_df = pd.read_sql("""
            SELECT 
                t.tag_name,
                COUNT(at.article_id) as article_count
            FROM tags t
            LEFT JOIN article_tags at ON t.tag_id = at.tag_id
            GROUP BY t.tag_name
            ORDER BY article_count DESC
        """, conn)
        
        conn.close()
        
        print("NORMALIZED DATA DASHBOARD")
        print("=" * 60)
        
        # Basic stats
        print(f"Overall Statistics:")
        print(f"   Total Articles: {len(articles_df)}")
        print(f"   Unique Sources: {articles_df['source_name'].nunique()}")
        print(f"   Unique Categories: {articles_df['category_name'].nunique()}")
        print(f"   Unique Tags: {len(tags_df)}")
        print(f"   Date Range: {articles_df['publication_date'].min()} to {articles_df['publication_date'].max()}")
        
        print(f"\nCategory Distribution:")
        category_counts = articles_df['category_name'].value_counts()
        for category, count in category_counts.items():
            percentage = (count / len(articles_df)) * 100
            print(f"   {category}: {count} articles ({percentage:.1f}%)")
        
        print(f"\nSource Distribution:")
        source_counts = articles_df['source_name'].value_counts().head(5)
        for source, count in source_counts.items():
            percentage = (count / len(articles_df)) * 100
            print(f"   {source}: {count} articles ({percentage:.1f}%)")
        
        print(f"\nContent Statistics:")
        print(f"   Average word count: {articles_df['word_count'].mean():.0f}")
        print(f"   Min word count: {articles_df['word_count'].min()}")
        print(f"   Max word count: {articles_df['word_count'].max()}")
        
        print(f"\nTop 10 Most Used Tags:")
        for i, (_, row) in enumerate(tags_df.head(10).iterrows(), 1):
            print(f"   {i}. {row['tag_name']}: {row['article_count']} articles")
        
        # Show latest articles
        print(f"\nLatest Articles:")
        latest = articles_df.nlargest(3, 'publication_date')
        for idx, row in latest.iterrows():
            print(f"   - {row['title'][:70]}...")
            print(f"     [{row['source_name']} - {row['category_name']} - {row['publication_date']}]")
            
    except Exception as e:
        print(f"Dashboard creation failed: {e}")

# Create dashboard
create_normalized_dashboard()

NORMALIZED DATA DASHBOARD
Overall Statistics:
   Total Articles: 7856
   Unique Sources: 1
   Unique Categories: 1
   Unique Tags: 0
   Date Range: 2019-02-15 to 2025-11-11

Category Distribution:
   unknown: 7856 articles (100.0%)

Source Distribution:
   dap-news.com: 7856 articles (100.0%)

Content Statistics:
   Average word count: 0
   Min word count: 0
   Max word count: 0

Top 10 Most Used Tags:

Latest Articles:
Dashboard creation failed: Column 'publication_date' has dtype object, cannot use method 'nlargest' with this dtype


C:\Users\Eii Lingg\AppData\Local\Temp\ipykernel_14440\1630780725.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  articles_df = pd.read_sql("""
C:\Users\Eii Lingg\AppData\Local\Temp\ipykernel_14440\1630780725.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tags_df = pd.read_sql("""


In [39]:
# %%
import psycopg2
import json
from datetime import datetime

# %%
DB_CONFIG = {
    'host': 'localhost',
    'database': 'mates',
    'user': 'postgres',
    'password': '1',
    'port': '5432'
}

JSON_FILE_PATH = r"d:\Menghour\MATES\scraping\notebook\khmer_news_formatted.json"

# %%
# STEP 1: Create normalized tables
def create_tables():
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    
    # Drop existing tables
    cur.execute("DROP TABLE IF EXISTS article_tags CASCADE")
    cur.execute("DROP TABLE IF EXISTS articles CASCADE") 
    cur.execute("DROP TABLE IF EXISTS tags CASCADE")
    cur.execute("DROP TABLE IF EXISTS sources CASCADE")
    cur.execute("DROP TABLE IF EXISTS categories CASCADE")
    
    # Create categories table
    cur.execute("""
        CREATE TABLE categories (
            category_id SERIAL PRIMARY KEY,
            category_name VARCHAR(100) UNIQUE NOT NULL,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    """)
    
    # Create sources table  
    cur.execute("""
        CREATE TABLE sources (
            source_id SERIAL PRIMARY KEY,
            source_url VARCHAR(500) UNIQUE NOT NULL,
            source_name VARCHAR(200) NOT NULL,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    """)
    
    # Create tags table
    cur.execute("""
        CREATE TABLE tags (
            tag_id SERIAL PRIMARY KEY,
            tag_name VARCHAR(100) UNIQUE NOT NULL,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    """)
    
    # Create articles table
    cur.execute("""
        CREATE TABLE articles (
            article_id SERIAL PRIMARY KEY,
            url VARCHAR(1000) UNIQUE NOT NULL,
            source_id INTEGER REFERENCES sources(source_id),
            publication_date DATE NOT NULL,
            scrape_date TIMESTAMP NOT NULL,
            title TEXT NOT NULL,
            content TEXT,
            word_count INTEGER DEFAULT 0,
            sentence_count INTEGER DEFAULT 0,
            character_count INTEGER DEFAULT 0,
            category_id INTEGER REFERENCES categories(category_id),
            category_confidence DECIMAL(3,2) DEFAULT 0.0,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    """)
    
    # Create article_tags junction table
    cur.execute("""
        CREATE TABLE article_tags (
            article_tag_id SERIAL PRIMARY KEY,
            article_id INTEGER REFERENCES articles(article_id) ON DELETE CASCADE,
            tag_id INTEGER REFERENCES tags(tag_id) ON DELETE CASCADE,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            UNIQUE(article_id, tag_id)
        )
    """)
    
    conn.commit()
    cur.close()
    conn.close()
    print("STEP 1: Tables created successfully")

create_tables()

# %%
# STEP 2: Load and inspect JSON data with error handling
def inspect_data():
    with open(JSON_FILE_PATH, 'r', encoding='utf-8') as f:
        articles = json.load(f)
    
    print(f"STEP 2: Loaded {len(articles)} articles from JSON")
    
    # Check first article structure with safe access
    first_article = articles[0]
    print("\nFirst article structure:")
    print(f"URL: {first_article.get('url', 'MISSING')}")
    print(f"Source: {first_article.get('source', 'MISSING')}")
    print(f"Primary Category: '{first_article.get('primary_category', 'MISSING')}'")
    print(f"Tags: {first_article.get('tags', [])}")
    print(f"Publication Date: {first_article.get('publication_date', 'MISSING')}")
    
    # Collect unique values with safe access
    sources = set()
    categories = set()
    all_tags = set()
    
    articles_without_category = 0
    
    for article in articles[:100]:  # Check first 100 articles
        # Source
        if article.get('source'):
            sources.add(article['source'])
        
        # Category - handle missing categories
        category = article.get('primary_category')
        if category:
            categories.add(category)
        else:
            articles_without_category += 1
            categories.add('unknown')  # Add default category
        
        # Tags
        for tag in article.get('tags', []):
            if tag:
                all_tags.add(tag)
    
    print(f"\nFound {articles_without_category} articles without category in first 100")
    print(f"Unique sources in first 100 articles: {list(sources)}")
    print(f"Unique categories in first 100 articles: {list(categories)}")
    print(f"Sample tags in first 100 articles: {list(all_tags)[:10]}")
    
    return articles

articles_data = inspect_data()

# %%
# STEP 3: Insert data in logical order
conn = psycopg2.connect(**DB_CONFIG)
cur = conn.cursor()

print("STEP 3: Starting data insertion...")

# Step 3.1: Insert sources
print("Step 3.1: Inserting sources...")
sources_inserted = set()

for article in articles_data:
    source_url = article.get('source')
    if source_url and source_url not in sources_inserted:
        source_name = source_url.replace('https://', '').replace('http://', '').split('/')[0]
        try:
            cur.execute(
                "INSERT INTO sources (source_url, source_name) VALUES (%s, %s)",
                (source_url, source_name)
            )
            sources_inserted.add(source_url)
            print(f"  Inserted source: {source_name}")
        except Exception as e:
            print(f"  Source already exists: {source_name}")

conn.commit()
print(f"Inserted {len(sources_inserted)} sources")

# Step 3.2: Insert categories (including 'unknown' for missing categories)
print("\nStep 3.2: Inserting categories...")
categories_inserted = set()

# First, ensure 'unknown' category exists for articles without category
try:
    cur.execute("INSERT INTO categories (category_name) VALUES ('unknown') ON CONFLICT DO NOTHING")
    categories_inserted.add('unknown')
except:
    pass

for article in articles_data:
    category_name = article.get('primary_category', 'unknown')
    if category_name and category_name not in categories_inserted:
        try:
            cur.execute(
                "INSERT INTO categories (category_name) VALUES (%s) ON CONFLICT DO NOTHING",
                (category_name,)
            )
            categories_inserted.add(category_name)
            print(f"  Inserted category: {category_name}")
        except Exception as e:
            print(f"  Category already exists: {category_name}")

conn.commit()
print(f"Inserted {len(categories_inserted)} categories")

# Step 3.3: Insert tags
print("\nStep 3.3: Inserting tags...")
tags_inserted = set()

for article in articles_data:
    for tag_name in article.get('tags', []):
        if tag_name and tag_name not in tags_inserted:
            try:
                cur.execute(
                    "INSERT INTO tags (tag_name) VALUES (%s) ON CONFLICT DO NOTHING",
                    (tag_name,)
                )
                tags_inserted.add(tag_name)
            except Exception as e:
                pass  # Tag already exists

conn.commit()
print(f"Inserted {len(tags_inserted)} tags")

# Step 3.4: Insert articles and link tags
print("\nStep 3.4: Inserting articles and linking tags...")
articles_inserted = 0
tags_linked = 0
errors = 0

for i, article in enumerate(articles_data):
    if i % 500 == 0:
        print(f"  Processing article {i}/{len(articles_data)}...")
    
    try:
        # Get source_id
        source_url = article.get('source')
        if not source_url:
            errors += 1
            continue
            
        cur.execute("SELECT source_id FROM sources WHERE source_url = %s", (source_url,))
        source_result = cur.fetchone()
        if not source_result:
            errors += 1
            continue
        source_id = source_result[0]
        
        # Get category_id - handle missing categories  
        category_name = article.get('primary_category', 'unknown')
        cur.execute("SELECT category_id FROM categories WHERE category_name = %s", (category_name,))
        category_result = cur.fetchone()
        category_id = category_result[0] if category_result else None
        
        # Convert dates
        pub_date_str = article.get('publication_date')
        scrape_date_str = article.get('scrape_date')
        if not pub_date_str or not scrape_date_str:
            errors += 1
            continue
            
        pub_date = datetime.strptime(pub_date_str, '%m/%d/%Y').date()
        scrape_date = datetime.strptime(scrape_date_str, '%m/%d/%Y %H:%M')
        
        # Insert article
        cur.execute("""
            INSERT INTO articles (
                url, source_id, publication_date, scrape_date, title, content,
                word_count, sentence_count, character_count, category_id, category_confidence
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT (url) DO NOTHING
            RETURNING article_id
        """, (
            article.get('url'), source_id, pub_date, scrape_date, article.get('title', ''),
            article.get('content', ''), article.get('word_count', 0), 
            article.get('sentence_count', 0), article.get('character_count', 0),
            category_id, article.get('category_confidence', 0.0)
        ))
        
        result = cur.fetchone()
        if result:
            article_id = result[0]
            articles_inserted += 1
            
            # Link tags to this article
            for tag_name in article.get('tags', []):
                if tag_name:
                    cur.execute("SELECT tag_id FROM tags WHERE tag_name = %s", (tag_name,))
                    tag_result = cur.fetchone()
                    if tag_result:
                        tag_id = tag_result[0]
                        try:
                            cur.execute(
                                "INSERT INTO article_tags (article_id, tag_id) VALUES (%s, %s) ON CONFLICT DO NOTHING",
                                (article_id, tag_id)
                            )
                            tags_linked += 1
                        except:
                            pass
        
        if i % 100 == 0:
            conn.commit()
            
    except Exception as e:
        errors += 1
        if errors < 10:  # Only print first 10 errors
            print(f"Error processing article {i}: {e}")
        continue

conn.commit()
print(f"Inserted {articles_inserted} articles")
print(f"Created {tags_linked} tag relationships")
print(f"Encountered {errors} errors")

cur.close()
conn.close()

# %%
# STEP 4: Verify the results
def verify_results():
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    
    print("\nSTEP 4: Verification Results")
    print("=" * 50)
    
    # Count records in each table
    cur.execute("SELECT COUNT(*) FROM articles")
    articles_count = cur.fetchone()[0]
    
    cur.execute("SELECT COUNT(*) FROM sources") 
    sources_count = cur.fetchone()[0]
    
    cur.execute("SELECT COUNT(*) FROM categories")
    categories_count = cur.fetchone()[0]
    
    cur.execute("SELECT COUNT(*) FROM tags")
    tags_count = cur.fetchone()[0]
    
    cur.execute("SELECT COUNT(*) FROM article_tags")
    article_tags_count = cur.fetchone()[0]
    
    print(f"Articles: {articles_count}")
    print(f"Sources: {sources_count}")
    print(f"Categories: {categories_count}") 
    print(f"Tags: {tags_count}")
    print(f"Article-Tag relationships: {article_tags_count}")
    
    # Show sources
    print("\nSources:")
    cur.execute("SELECT source_id, source_url, source_name FROM sources")
    for source_id, source_url, source_name in cur.fetchall():
        print(f"  {source_id}: {source_name} ({source_url})")
    
    # Show categories with article counts
    print("\nCategories with article counts:")
    cur.execute("""
        SELECT c.category_name, COUNT(a.article_id) 
        FROM categories c 
        LEFT JOIN articles a ON c.category_id = a.category_id 
        GROUP BY c.category_name 
        ORDER BY COUNT(a.article_id) DESC
    """)
    for category_name, count in cur.fetchall():
        print(f"  {category_name}: {count} articles")
    
    # Show top tags
    print("\nTop 10 tags:")
    cur.execute("""
        SELECT t.tag_name, COUNT(at.article_id) 
        FROM tags t 
        JOIN article_tags at ON t.tag_id = at.tag_id 
        GROUP BY t.tag_name 
        ORDER BY COUNT(at.article_id) DESC 
        LIMIT 10
    """)
    for tag_name, count in cur.fetchall():
        print(f"  {tag_name}: {count} articles")
    
    # Show sample articles with their categories and tags
    print("\nSample articles with categories and tags:")
    cur.execute("""
        SELECT a.article_id, a.title, c.category_name, a.url
        FROM articles a
        LEFT JOIN categories c ON a.category_id = c.category_id
        ORDER BY a.publication_date DESC
        LIMIT 3
    """)
    for article_id, title, category, url in cur.fetchall():
        print(f"\n  Article {article_id}:")
        print(f"    Title: {title[:60]}...")
        print(f"    Category: {category}")
        print(f"    URL: {url}")
        
        # Get tags for this article
        cur.execute("""
            SELECT t.tag_name 
            FROM tags t
            JOIN article_tags at ON t.tag_id = at.tag_id
            WHERE at.article_id = %s
        """, (article_id,))
        tags = [tag[0] for tag in cur.fetchall()]
        print(f"    Tags: {tags}")
    
    cur.close()
    conn.close()

verify_results()

STEP 1: Tables created successfully
STEP 2: Loaded 7856 articles from JSON

First article structure:
URL: https://dap-news.com/international/2022/03/29/232389/
Source: https://dap-news.com
Primary Category: 'MISSING'
Tags: []
Publication Date: 03/29/2022

Found 100 articles without category in first 100
Unique sources in first 100 articles: ['https://dap-news.com']
Unique categories in first 100 articles: ['unknown']
Sample tags in first 100 articles: []
STEP 3: Starting data insertion...
Step 3.1: Inserting sources...
  Inserted source: dap-news.com
Inserted 1 sources

Step 3.2: Inserting categories...
Inserted 1 categories

Step 3.3: Inserting tags...
Inserted 0 tags

Step 3.4: Inserting articles and linking tags...
  Processing article 0/7856...
  Processing article 500/7856...
  Processing article 1000/7856...
  Processing article 1500/7856...
  Processing article 2000/7856...
  Processing article 2500/7856...
  Processing article 3000/7856...
  Processing article 3500/7856...
  Pr